Using: 

https://medium.com/future-vision/google-maps-in-python-part-2-393f96196eaf

https://buildmedia.readthedocs.org/media/pdf/jupyter-gmaps/latest/jupyter-gmaps.pdf

In [64]:
import os
import pandas as pd
import gmaps
from datetime import datetime
import googlemaps

In [65]:
# create function to read in api key from txt file
def read_in_google_api_key(file_directory):
    api_key = pd.read_csv("gmaps_api_key.txt", header = None).iloc[0, 0]
    return api_key

In [88]:
def return_cityState_from_longitudeLatitude(long, lat):
    return googlemaps_client.reverse_geocode((long, lat))[0]['formatted_address']

### Setup Key

In [66]:
# read in api key from txt file
google_api_key = read_in_google_api_key(os.getcwd())

In [67]:
# setup api key for gmaps api
gmaps.configure(api_key=google_api_key)

In [68]:
# setup api key for GoogleMaps api
googlemaps_client = googlemaps.Client(key = google_api_key)

### Setup two example locations

In [70]:
location1 = (googlemaps_client.geocode("Bloomington, IL")[0]["geometry"]['location']['lat'], 
             googlemaps_client.geocode("Bloomington, IL")[0]["geometry"]['location']['lng'])

In [71]:
location2 = (googlemaps_client.geocode("Cleveland, OH")[0]["geometry"]['location']['lat'], 
             googlemaps_client.geocode("Cleveland, OH")[0]["geometry"]['location']['lng'])

### Trip Logistics

In [184]:
def create_optimal_trip_directions(googlemaps_client, location1, location2):
    direction_route_info = googlemaps_client.directions(location1, location2, mode = "driving")[0]
    
    # distance of entire trip
    trip_distance = direction_route_info['legs'][0]['distance']['text']
    print("trip distance: "+str(trip_distance))
    # duration of entire trip
    trip_duration = direction_route_info['legs'][0]['duration']['text']

In [129]:
def create_route_waypoints(direction_route_info):    
    
    

In [130]:
direction_route_info['legs'][0]['steps']

[{'distance': {'text': '26 ft', 'value': 8},
  'duration': {'text': '1 min', 'value': 1},
  'end_location': {'lat': 40.4842497, 'lng': -88.99377009999999},
  'html_instructions': 'Head <b>northwest</b> on <b>N East St</b>',
  'polyline': {'points': 'eabvFxqd~OKF'},
  'start_location': {'lat': 40.4841869, 'lng': -88.99372939999999},
  'travel_mode': 'DRIVING'},
 {'distance': {'text': '3.6 mi', 'value': 5807},
  'duration': {'text': '9 mins', 'value': 528},
  'end_location': {'lat': 40.5364073, 'lng': -88.9958843},
  'html_instructions': 'Continue onto <b>N Main St</b><div style="font-size:0.9em">Pass by Noodles and Company (on the left in 1.4&nbsp;mi)</div>',
  'polyline': {'points': 'qabvF`rd~OMDIBI@I@M@{EJ]@q@@iCBs@@w@@eCDi@@e@@sABoA@e@?uBBs@@aA@U?s@?m@@o@@oA@M@I?qA@m@@c@@{@Bg@@I?K?S?O@K?kBBeFHg@@a@@kBDo@@uBBU?aEHiBBcDF{CFoCBoABgAB_ABqABk@?O?m@@_ABk@@wA@W@k@@]?gAB_@@sCBcABa@?qBF_A@I?M?K@[?_ABQ?_@@]@e@?O@_B@M@]?qAB}AD]?_B@Q?A?E?E?A?O?G?G@A?M?gB@w@BQ?G?q@?C?i@?cA@c@@i@@i@?c@?{@@c@?_@?UA

### Stops/Waypoints

In [187]:
df_tripDirections = pd.DataFrame(columns = ['start_location_lat', 'start_location_lng', 'end_location_lat', 'end_location_lng', 
                                            'distance_value', 'distance_text', 'duration_value', 'duration_text'])
i = 0
#these are the steps of the trip, my goal is to show the weather at each of them, (every other?)
for direction in direction_route_info['legs'][0]['steps']:

    df_tripDirections.loc[i] = [direction['start_location']['lat'], direction['start_location']['lng'], 
                                direction['end_location']['lat'], direction['end_location']['lng'], 
                                direction['distance']['value'], direction['distance']['text'], 
                                direction['duration']['value'], direction['duration']['text']]
    i += 1

In [188]:
import re
re.sub("[^0-9]", "", "sdkjh987978asd098as0980a98sd")

'987978098098098'

In [189]:
df_tripDirections['cumulative_distance_miles'] = df_tripDirections['distance_text'].str.replace(r"[a-zA-Z]", '').astype('float').cumsum()

In [193]:
import math

In [199]:
(df_tripDirections['cumulative_distance_miles']/60).astype(int)

0      0
1      0
2      1
3      2
4      2
5      2
6      2
7      9
8      9
9     14
10    14
11    15
12    15
13    15
14    15
15    15
16    15
17    15
18    15
19    20
Name: cumulative_distance_miles, dtype: int32

In [172]:
df_tripWaypoint_locations = df_tripDirections[df_tripDirections['distance_value'] >= 40000]
df_tripWaypoint_locations['end_location_text'] = df_temp.apply(lambda x: return_cityState_from_longitudeLatitude(x['end_location_lat'], x['end_location_lng']), axis = 1)

C:\Users\David\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [173]:
df_tripWaypoint_locations

,start_location_lat,start_location_lng,end_location_lat,end_location_lng,distance_value,distance_text,duration_value,duration_text,end_location_text
2,40.536407,-88.995884,41.479901,-88.197113,136879,85.1 mi,4780,1 hour 20 mins,"22140 SW Frontage Rd, Joliet, IL 60404, USA"
9,41.584602,-87.234983,41.386756,-82.178565,447806,278 mi,14966,4 hours 9 mins,"I-90, Elyria, OH 44035, USA"
10,41.386756,-82.178565,41.474452,-81.723184,43405,27.0 mi,1540,26 mins,"2100 W 53rd St, Cleveland, OH 44102, USA"


In [ ]:
dict_temp = {"name": }

In [165]:
def create_custom_markers_from_directions(df_tripWaypoint_locations):
    list_custom_waypoint_dictionaries = []
    for i in range(len(df_tripWaypoint_locations)):
        temp_waypoint_location = df_tripWaypoint_locations.iloc[i, :]
        dict_custom_waypoint_info = {"name": temp_waypoint_location['end_location_text'],
                                    "location": (temp_waypoint_location['end_location_lat'], temp_waypoint_location['end_location_lng']),
                                    "weather_degrees": "TBD",
                                    "weather_info": "TBD"}
        list_custom_waypoint_dictionaries.append(dict_custom_waypoint_info)
    
    info_box_template = """
        <dl>
        <dt>Location</dt><dd>{name}</dd>
        <dt>Weather (Degrees)</dt><dd>{weather_degrees}</dd>
        <dt>Weather (Info)</dt><dd>{weather_info}</dd>
        </dl>
    """
    custom_waypoint_locations = [waypoint['location'] for waypoint in list_custom_waypoint_dictionaries]
    custom_waypoint_info = [info_box_template.format(**waypoint) for waypoint in list_custom_waypoint_dictionaries]
    
    return custom_waypoint_locations, custom_waypoint_info

In [167]:
custom_waypoint_locations, custom_waypoint_info = create_custom_markers_from_directions(df_tripWaypoint_locations)

### Plot

In [169]:
#Create the map
fig = gmaps.figure()
#create the layer
layer = gmaps.directions.Directions(location1, location2, mode='car')
#Add the layer
fig.add_layer(layer)
custom_markers = gmaps.marker_layer(custom_waypoint_locations, info_box_content=custom_waypoint_info)
fig.add_layer(custom_markers)
fig

Figure(layout=FigureLayout(height='420px'))

### Test Area

In [147]:
temp = pd.DataFrame({"a": [1, 2, 3], "b": [56, 2, 645]})

In [151]:
for i in range(len(temp)):
    print(temp.loc[i])

a     1
b    56
Name: 0, dtype: int64
a    2
b    2
Name: 1, dtype: int64
a      3
b    645
Name: 2, dtype: int64


In [149]:
temp

,a,b
0,1,56
1,2,2
2,3,645
